In [2]:
import pickle
import dotenv

In [3]:
import tqdm
import time
import re

In [4]:
from mmw.parse import *

In [5]:
dotenv.load_dotenv()

True

In [ ]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response['choices'][0]['text']

In [ ]:
questions_explanation = parse_question_explanations()

In [ ]:
all_parsed_essays = parse_essays()

In [ ]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [ ]:
len(all_parsed_essays), len(filtered_marked_essays)

In [ ]:
model = "text-davinci-003"

#models simple/ cheap/ fast to advanced:
#model = "ada"
#model = "babbage"
#model = "curie"
#model = "davinci"

#model = "ada:ft-mark-my-words-2023-05-01-11-44-26"

In [ ]:
LIMIT_ESSAYS = 20

In [ ]:
all_prompts = assemble_prompts(all_parsed_essays, filtered_marked_essays, questions_explanation, template=PROMPT_TPLS[1])

In [ ]:
all_answers = []
for prompt_array in tqdm.tqdm(all_prompts[:LIMIT_ESSAYS*len(questions_explanation)]):
    i, j, prompt, annot = prompt_array
    response = openai.Completion.create(model=model, prompt=prompt, temperature=0, max_tokens=3)
    all_answers.append((i, j, prompt, response, annot))
    
    time.sleep(.5)

In [ ]:
len(all_answers)

In [ ]:
all_answers[0]

In [ ]:
save_results('data/evaluations/run_4_text-davinci_explanation', all_parsed_essays, all_questions, all_answers)

In [6]:
_,_,all_answers_loaded = load_results('data/evaluations/run_4_text-davinci_explanation')

In [8]:
all_answers_loaded[0][2]

'Given is a student essay and a question to the essay. Please rate the Question with a number between 0 and 3, where 0 is "not at all satisfied" and 3 is "very satisfied.".\n\nEssay: """\nThe Cage\nIt felt like I was walking forever, i didn\'t even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is rain and i was not gonna get my new guchi shoes wet so i bulted to the end of the crusty aliway,\ni saw... well i didn\'t see anything other than cages, well i guess it was just a dream cause i don\'t own\nguchi shoes. OK enough of me being broke I gotta get to school. Time forwords and

In [7]:
[(a[0], a[1], a[-1], a[3]['choices'][0]['text']) for a in all_answers_loaded]

[(0, 0, 1, ' 3'),
 (0, 1, 1, ' 3'),
 (0, 2, 1, ' 3'),
 (0, 3, 1, ' 3'),
 (0, 4, 1, ' 3'),
 (0, 5, 0, ' 3'),
 (0, 6, 0, ' 3'),
 (0, 7, 0, ' 3'),
 (0, 8, 0, ' 3'),
 (0, 9, 0, ' 3'),
 (0, 10, 0, ' 3'),
 (0, 11, 0, ' 3'),
 (0, 12, 0, ' 3'),
 (1, 0, 2, ' 3'),
 (1, 1, 2, ' 3'),
 (1, 2, 3, ' 3'),
 (1, 3, 3, ' 3'),
 (1, 4, 2, ' 3'),
 (1, 5, 3, ' 3'),
 (1, 6, 3, ' 3'),
 (1, 7, 2, ' 3'),
 (1, 8, 3, ' 3'),
 (1, 9, 2, ' 3'),
 (1, 10, 2, ' 3'),
 (1, 11, 3, ' 3'),
 (1, 12, 3, ' 3'),
 (2, 0, 3, ' 3'),
 (2, 1, 3, ' 3'),
 (2, 2, 3, ' 3'),
 (2, 3, 3, ' 3'),
 (2, 4, 3, ' 3'),
 (2, 5, 3, ' 3'),
 (2, 6, 3, ' 3'),
 (2, 7, 3, ' 3'),
 (2, 8, 3, ' 3'),
 (2, 9, 3, ' 3 - Very'),
 (2, 10, 3, ' 3'),
 (2, 11, 3, ' 3'),
 (2, 12, 1, ' 3'),
 (3, 0, 1, ' 3'),
 (3, 1, 3, ' 3'),
 (3, 2, 3, ' 3'),
 (3, 3, 3, ' 3'),
 (3, 4, 3, ' 3'),
 (3, 5, 3, ' 3'),
 (3, 6, 3, ' 3'),
 (3, 7, 3, ' 3'),
 (3, 8, 3, ' 3'),
 (3, 9, 3, ' 3'),
 (3, 10, 3, ' 3'),
 (3, 11, 3, ' 3'),
 (3, 12, 2, ' 3'),
 (4, 0, 2, ' 3'),
 (4, 1, 2, ' 3'),
 (4, 2, 2